In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

LIBRARY IMPORT

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os

In [ ]:
import torch

In [ ]:
!pip install transformers

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [ ]:
device = torch.device('cpu')



```
# Выбран кодовый формат
```

DATA PREPARATION

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
original_path = '/content/gdrive/My Drive/Datasets/'

In [ ]:
data_dir = os.path.join(original_path)
data_dir

In [ ]:
df_url=os.path.join(data_dir, 'big_data.json')


In [ ]:
data=pd.read_json(df_url)

In [ ]:
dic={
     'element':[],
     
     
}
df=pd.DataFrame(dic)
df


In [ ]:
for k,i in enumerate(data.rows.copy()):
  df.loc[k]=[i['element'], ]

In [ ]:
for i in df['element']:
    del i['address']

In [ ]:
df.head()

In [ ]:
for k,i in enumerate(data.rows.copy()):
  df['element'][k]=" ".join(sorted(i['element'].values()))

In [ ]:
df.head()

TEXT GENERATION

In [ ]:
!pip install sentencepiece

In [ ]:
model = T5ForConditionalGeneration.from_pretrained('t5-small')
device = torch.device('cpu')

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("t5-small")


In [ ]:
df.loc[16549][0]

In [ ]:
tokenized_data=[]
for  i in df['element'][:]:
      tokenized_data.append(tokenizer.encode(i,return_tensors="pt"))

In [ ]:
tokenized_text=tokenized_data.copy()

In [ ]:
preds=[]

In [ ]:
# summmarize 
summary_ids = model.generate(tokenized_data[4],
                                    num_beams=4,
                                    no_repeat_ngram_size=2,
                                    min_length=50,
                                    max_length=100,
                                    early_stopping=True)

output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [ ]:
df.head()

In [ ]:
df.loc[4][0]

In [ ]:
output

In [ ]:
# summmarize 
preds=[]
for i in range(100):
    summary_ids = model.generate(tokenized_data[i],
                                    num_beams=4,
                                    no_repeat_ngram_size=2,
                                    min_length=30,
                                    max_length=100,
                                    early_stopping=True)

    preds.append(tokenizer.decode(summary_ids[0], skip_special_tokens=True))

In [ ]:
len(preds)

In [ ]:
preds

In [ ]:
df2=df[:100].copy()

In [ ]:
df2['description']=preds

In [ ]:
df2.head()

In [ ]:
df2.to_csv('element_2_description.csv')